<a href="https://colab.research.google.com/github/tjayant/Predix-HelloWorld-WebApp/blob/master/ExploringWord2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XCS224N Assignment 1 Part 2: Exploring Word2Vec Embeddings (15 Points)

Before you start, make sure you read the XCS224N Assignment 1 PDF Handout.

As you work you way through this notebook, you will enter answers directly into a companion quiz linked in the SCPD learning portal. The name of this quiz is **Assignment 1 Part 2 Companion Quiz**. Log in to the SCPD site now and open the quiz before you begin. 

In [0]:
# All Import Statements Defined Here
# Note: Do not add to this list.
# All the dependencies you need can be installed by running this cell.
# Throughout this notebook you can run a cell by hitting CTRL+RETURN or the Play button/icon at left
# ----------------

import sys
assert sys.version_info[0]==3
assert sys.version_info[1] >= 5

from gensim.models import KeyedVectors
from gensim.test.utils import datapath
import pprint
import matplotlib.pyplot as plt

# ----------------

As discussed in class, more recently prediction-based word vectors have come into fashion, e.g. word2vec. Here, we shall explore the embeddings produced by word2vec. Please revisit the class notes and lecture slides for more details on the word2vec algorithm. If you're feeling adventurous, challenge yourself and try reading the [original paper](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf).

Then run the following cells to load the word2vec vectors into memory. **_Note: This could take several minutes._**

In [0]:
def load_word2vec():
    """ Load Word2Vec Vectors
        Return:
            wv_from_bin: 2.5 million of 3 million embeddings, each lengh 300
    """
    import gensim.downloader as api
    from gensim.models import KeyedVectors
    # let's load 2.5 million of the 3 million word embeddings so we don't run out of memory on Colab
    wv_from_bin = KeyedVectors.load_word2vec_format(api.load("word2vec-google-news-300", return_path=True), limit=2500000, binary=True)
    vocab = list(wv_from_bin.vocab.keys())
    print("Loaded vocab size %i" % len(vocab))
    return wv_from_bin

In [0]:
# -----------------------------------
# Run Cell to Load Word Vectors
# Note: This may take several minutes
# -----------------------------------
wv_from_bin = load_word2vec()

###Question 1 (2 points)

The first question relates to the plot from the first part of the assignment. Please visit the Assignment 1 Part 2 Companion quiz in the SCPD learning platform, and answer this question prior to continuing. 





### Cosine Similarity
Now that we have word vectors, we need a way to quantify the similarity between individual words, according to these vectors. One such metric is cosine-similarity. We will be using this to find words that are "close" and "far" from one another.

We can think of n-dimensional vectors as points in n-dimensional space. If we take this perspective L1 and L2 Distances help quantify the amount of space "we must travel" to get between these two points. Another approach is to examine the angle between two vectors. From trigonometry we know that:

<img src="https://drive.google.com/uc?id=1PEmKQxxs5XB-N1Hz-jr514iWoqa62MgL" width=20% style="float: center;"></img>

Instead of computing the actual angle, we can leave the similarity in terms of $similarity = cos(\Theta)$. Formally the [Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity) $s$ between two vectors $p$ and $q$ is defined as:

$$s = \frac{p \cdot q}{||p|| ||q||}, \textrm{ where } s \in [-1, 1] $$ 

### Questions 2-10: Homonymns and Similarity (4 points)

Homonyms are words with more than one meaning. We want to see how our word embeddings capture this phenomenon for such words. We are going to test if, for certain homonyms, the top-10 most similar words (according to cosine similarity) contain related words from *both* meanings. For example, "leaves" has both "vanishes" and "stalks" in the top 10, and "scoop" has both "handed_waffle_cone" and "lowdown". 

**Note**: You should use the `wv_from_bin.most_similar(word)` function to get the top 10 similar words. This function ranks all other words in the vocabulary with respect to their cosine similarity to the given word. For further assistance please check the __[GenSim documentation](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.FastTextKeyedVectors.most_similar)__.


In [0]:
### Run this cell to print out the Top-10 most similar words
### Try the sample and then use this cell to complete Questions 2-10 in the companion quiz 

wv_from_bin.most_similar("mole")


### Questions 11-19: Synonyms & Antonyms (4 points) 

When considering Cosine Similarity, it's often more convenient to think of Cosine Distance, which is simply (1 - Cosine Similarity).

We will look for triplets of words (w1,w2,w3) where w1 and w2 are synonyms and w1 and w3 are antonyms, but Cosine Distance(w1,w3) < Cosine Distance(w1,w2). For example, w1="happy" is closer to w3="sad" than to w2="cheerful".

You should use the the `wv_from_bin.distance(w1, w2)` function here in order to compute the cosine distance between two words. Please see the __[GenSim documentation](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.FastTextKeyedVectors.distance)__ for further assistance.


In [0]:
### Run this cell to compare cosine distances between synonyms and antonyms
### Try the sample and then use this cell to complete Questions 11-19 in the companion quiz

w1 = "happy"
w2 = "cheerful"
w3 = "sad"
w1_w2_dist = wv_from_bin.distance(w1, w2)
w1_w3_dist = wv_from_bin.distance(w1, w3)

print("Synonyms {}, {} have cosine distance: {}".format(w1, w2, w1_w2_dist))
print("Antonyms {}, {} have cosine distance: {}".format(w1, w3, w1_w3_dist))


### Questions 20-31: Analogies with Word Vectors (3 Points)
Word2Vec vectors have been shown to *sometimes* exhibit the ability to solve analogies. 

As an example, for the analogy "man : king :: woman : x", what is x?

In the cell below, we show you how to use word vectors to find x. The `most_similar` function finds words that are most similar to the words in the `positive` list and most dissimilar from the words in the `negative` list. **The model's proposed answer to the analogy will be the word ranked most similar (largest numerical value).**

We will test whether or not the word vectors can solve a handful of pre-selected analogies. 

**Note:** Further Documentation on the `most_similar` function can be found within the __[GenSim documentation](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.FastTextKeyedVectors.most_similar)__.

In [0]:
# Run this cell to answer the analogy -- man : king :: woman : x
# Try the sample and then use this cell to complete Questions 20-31 in the companion quiz

pprint.pprint(wv_from_bin.most_similar(positive=['woman', 'king'], negative=['man']))

### Question 32: Guided Analysis of Bias in Word Vectors [written] (2 point)

It's important to be cognizant of the biases (gender, race, sexual orientation etc.) implicit in our word embeddings. Execute the cells bellow and answer the final multiple choice question. 

Run the cell below, to examine (a) which terms are most similar to "woman" and "doctor" and most dissimilar to "man", and (b) which terms are most similar to "man" and "doctor" and most dissimilar to "woman".

In [0]:
# Run this cell
# Here `positive` indicates the list of words to be similar to and `negative` indicates the list of words to be
# most dissimilar from.
pprint.pprint(wv_from_bin.most_similar(positive=['woman', 'doctor'], negative=['man']))
print()
pprint.pprint(wv_from_bin.most_similar(positive=['man', 'doctor'], negative=['woman']))

Now, as another example, run the cell below. 

In [0]:

pprint.pprint(wv_from_bin.most_similar(positive=['brown', 'driving'], negative=['white']))
print()
pprint.pprint(wv_from_bin.most_similar(positive=['white', 'driving'], negative=['brown']))


# <font color="blue"> Submission Instructions</font>

Simply make sure you have answered all the multiple choice questions to your satisfaction, and submit!
  